# Over representation  analysis and gene set enrichment analysis 

In [1]:
import pandas as pd
qvalues = pd.read_csv("../multiplehypo/qvalues.tsv",sep="\t",index_col=0)

In [2]:
qvalues.loc[qvalues["q"]<1e-15,"q"]

Hugo_Symbol
NAT1       7.780762e-42
TBC1D9     2.475169e-40
ESR1       2.475169e-40
ARMT1      3.624451e-40
SLC16A6    3.707353e-40
               ...     
ATP11A     9.362331e-16
FAM188B    9.389268e-16
LCA5L      9.418759e-16
AFF4       9.448362e-16
PDHB       9.621346e-16
Name: q, Length: 775, dtype: float64

In [9]:
import gseapy as gp

pathway_db=['KEGG_2019_Human']
background=qvalues.index
gene_list = list(qvalues.loc[qvalues["q"]<1e-15,"q"].index)

output_enrichr=pd.DataFrame()
enr=gp.enrichr(
                gene_list=gene_list,
                gene_sets=pathway_db,
                background=background
            )


In [25]:
kegg_enr = enr.results[["P-value","Overlap","Term"]].rename(columns={"P-value": "p"})

In [26]:
import numpy as np
import numpy.random as npr

def bootstrap(invec):
    idx = npr.randint(0, len(invec), len(invec))
    return [invec[i] for i in idx]

def estimatePi0(p, numBoot=100, numLambda=100, maxLambda=0.95):
    p.sort()
    n=len(p)
    lambdas=np.linspace(maxLambda/numLambda,maxLambda,numLambda)
    Wls=np.array([n-np.argmax(p>=l) for l in lambdas])
    pi0s=np.array([Wls[i] / (n * (1 - lambdas[i])) for i in range(numLambda)])
    minPi0=np.min(pi0s)
    mse = np.zeros(numLambda)
    for boot in range(numBoot):
        pBoot = bootstrap(p)
        pBoot.sort()
        WlsBoot =np.array([n-np.argmax(pBoot>=l) for l in lambdas])
        pi0sBoot =np.array([WlsBoot[i] / (n *(1 - lambdas[i])) for i in range(numLambda)])
        mse = mse + np.square(pi0sBoot-minPi0)
    minIx = np.argmin(mse)
    return pi0s[minIx]

def qvalues(pvalues):
    m = pvalues.shape[0] # The number of p-values
    pvalues.sort_values("p",inplace=True) # sort the pvalues in acending order
    pi0 = estimatePi0(list(pvalues["p"].values))
    print("pi_0 estimated to " + str(pi0))
    
    # calculate a FDR(t) as in Storey & Tibshirani
    num_p = 0.0
    for ix in pvalues.index:
        num_p += 1.0
        fdr = pi0*pvalues.loc[ix,"p"]*m/num_p
        pvalues.loc[ix,"q"] = fdr
    
    # calculate a q(p) as the minimal FDR(t)
    old_q=1.0
    for ix in reversed(list(pvalues.index)):
        q = min(old_q,pvalues.loc[ix,"q"])
        old_q = q
        pvalues.loc[ix,"q"] = q
    return pvalues

In [43]:
import scipy.stats as stats
pvalue = stats.hypergeom.sf(24,  len(background), 124, len(gene_list))
pvalue

6.797034073626332e-08

In [28]:
kegg_enr = qvalues(kegg_enr)

pi_0 estimated to 0.9464705036660755


In [30]:
kegg_enr.loc[kegg_enr["q"]<0.05]

,p,Overlap,Term,q
0,6.142575e-11,24/124,Cell cycle,1.575531e-08
1,1.631457e-04,17/160,Cellular senescence,2.092291e-02
2,3.322946e-04,7/35,"Alanine, aspartate and glutamate metabolism",2.841047e-02
